## Evaluations Model Responses using _LLM as a judge_
---

This notebook does as follows:

1. Reads all the responses from the previous inference step and runs evaluations on the responses using an _LLM as a judge_ that selects the best model, corresponding best response given the question and context, and the subjective evaluation/explanation for choosing that model.

1. Records metrics like the `p90, p95` latency, as well as `explanation` files as to why a given model was selected by the _LLM as a judge_ and why other's were not based on correctness and relevancy.

1. Uses a _Final LLM as a summarizer_ to parse through all of the subjective evaluations/explanations provided by the _LLM as a judge_ and gives a final analysis on the trends, patterns spotted across the model performance and gives a summary of which model is preferred for a given use case/dataset

*The model to be used as a judge and the final analysis summarizer can be configured in the `llm_as_a_judge_info` and the `final_analysis_summarizer` sections in the [config.yaml](config.yaml) file.*

In [1]:
# import the libraries
import os
import re
import ray
import json
import glob
import yaml
import time
import boto3
import logging
import botocore
import pandas as pd
from pathlib import Path
from functools import reduce
from litellm import completion
from typing import Dict, List, Optional

In [2]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# initialize the ray service to run async calls in parallel to bedrock easily
if ray.is_initialized():
    ray.shutdown()
ray.init()

2024-06-08 11:05:57,667	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.10.0


In [4]:
# global constants
CONFIG_FILE_PATH = "config.yaml"

# read the config yaml file
fpath = CONFIG_FILE_PATH
with open(fpath, 'r') as yaml_in:
    config = yaml.safe_load(yaml_in)
logger.info(f"config read from {fpath} -> {json.dumps(config, indent=2)}")

[2024-06-08 11:06:04,728] p23556 {2927026569.py:8} INFO - config read from config.yaml -> {
  "app_name": "llm-as-a-judge-eval-pipeline",
  "aws": {
    "region": "us-east-1"
  },
  "run_steps": {
    "1_get_inference.ipynb": true,
    "2_run_llm_as_a_judge_eval.ipynb": true
  },
  "dir_info": {
    "data_dir": "data",
    "dataset_dir": "source_data",
    "dataset_file_name": "data.csv",
    "metrics": "results",
    "llm_as_a_judge_dir": "eval_completions",
    "prompt_dir": "prompt_template",
    "llm_as_a_judge_completions": "llm_as_a_judge_completions.csv",
    "raw_llm_as_a_judge_completions": "raw_llm_responses.csv",
    "llm_as_a_judge_comparisons": "llm_as_a_judge_comparisons.csv",
    "llm_comparisons_txt": "llm_as_a_judge_comparisons.txt",
    "llm_as_a_judge_pick_rate": "llm_as_a_judge_pick_rate.csv",
    "eval_prompt_template": "llama3_eval_prompt.txt",
    "prompt_template": "prompt_template.txt",
    "processed_eval_prompts": "processed_eval_prompts.csv",
    "inference_

In [5]:
# initialize all global variables that are used across this notebook hydrated from the `config.yaml` file

# name of your csv file (containing the dataframe)
FILE_NAME: str = config['dir_info']['dataset_file_name']
# data directory
DATA_DIR: str = config['dir_info']['data_dir']

# result files
INFERENCE_LATENCY_SUMMARY_FPATH = os.path.join(DATA_DIR, config['dir_info']['inference_latency_summary_fname'])
METRICS_DIR: str = os.path.join(DATA_DIR, config['dir_info'] ['metrics'])
JSON_TXT_FILE_PATH: str = os.path.join(METRICS_DIR, config['dir_info']['llm_comparisons_txt'])
ALL_EXPLANATIONS_FPATH: str = os.path.join(METRICS_DIR, config['dir_info']['all_explanations'])
FINAL_ANALYSIS_MODEL_ID: str = config['final_analysis_summarizer']
FINAL_SUMMARY_ANALYSIS: str = os.path.join(METRICS_DIR, config['dir_info']['final_summary_analysis'])
bedrock_model_ids: List[str] = config['bedrock_fms_to_test']
USER_PROMPT_COL: str = config['dataset_info']['user_question_col']
SYSTEM_PROMPT_COL: str = config['dataset_info']['system_prompt_col']
INFERENCE_PARAMETERS: Dict = config['inference_parameters']
ON_LIST = list(filter(None, [USER_PROMPT_COL, 
                             SYSTEM_PROMPT_COL]))

### Use _LLM as a Judge_ Evaluations
---
In this portion:

1. Responses generated by each model are evaluated on relevance and meaning by your model of choice that acts as a `Judge`. Prompt for the model that acts as a judge be viewed and tweaked for different use cases in the: [prompt_template/](prompt_template/) directory. Edit and review this prompt based on the use case and criteria for subjective evaluation.

1. The role of the model acting as a judge it to compare the responses generated by each model and the already provided responses in the source dataset (if any). It provides information on the selected model, response, and an explanation of its selection, with a detailed analysis of comparison between other responses and why it chose the one it did.

*Note: For more information on the use of having a Model act as a judge, view: https://huggingface.co/learn/cookbook/en/llm_judge*

In [6]:
def prepare_eval_prompts(row):
    """
    This function evaluates the prompts by incorporating all of the responses generated by various models into the evaluation prompt template.
    """
    eval_template: Optional[str] = None
    processed_eval_template: Optional[str] = None
    model_responses: List[str] = []
    try:
        # file path to the eval template
        eval_template_path: str = config['llm_as_a_judge_info']['prompt_template']
        with open(eval_template_path, "r") as f:
            eval_template = f.read()
            logger.info(f"evaluation prompt template recorded: {eval_template}")
    except FileNotFoundError:
        print(f"Error: Evaluation template not found at {eval_template_path}")
    for column in row.index:
        if column.endswith("-response") and column != config['dataset_info']['pre_existing_response_col']:
            model_id = column.split("-response")[0]
            model_response = row[column]
            model_responses.append(f"\n<{model_id}>\n{model_response}\n</{model_id}>\n")
    print(f"model_responses: {model_responses}")

    if config['dataset_info']['system_prompt_col'] is not None:
        # if the system prompt is provided in the dataset, it is used as context
        processed_eval_template = eval_template.format(
            context=row[config['dataset_info']['system_prompt_col']], 
            question=row[config['dataset_info']['user_question_col']], 
            original_answer=row[config['dataset_info']['pre_existing_response_col']],
            model_responses="\n".join(model_responses)
        )
    else:
        # if the system prompt is not provided, the user column is assumed to have the context and so 
        # all the context is fit into the question itself
        processed_eval_template = eval_template.format(
            context=" ", 
            question=row[config['dataset_info']['user_question_col']], 
            original_answer=row[config['dataset_info']['pre_existing_response_col']],
            model_responses="\n".join(model_responses)
        )
    return processed_eval_template

#### Retrieve all the results from the `results.csv` file generated in the _Inference Step_

In [7]:
# Read the inference results
inference_results_file: str = os.path.join(METRICS_DIR, 
                                           config['dir_info']['all_results_file_name'])
df_resp_all = pd.read_csv(inference_results_file)
df_resp_all.head(10)

,user_input,anthropic.claude-3-haiku-20240307-v1:0-response,anthropic.claude-3-sonnet-20240229-v1:0-response,anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds,anthropic.claude-3-haiku-20240307-v1:0-prompt_token_count,anthropic.claude-3-haiku-20240307-v1:0-completion_token_count,anthropic.claude-3-haiku-20240307-v1:0-exception,anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds,anthropic.claude-3-sonnet-20240229-v1:0-prompt_token_count,anthropic.claude-3-sonnet-20240229-v1:0-completion_token_count,anthropic.claude-3-sonnet-20240229-v1:0-exception,model_1
0,Human: You are an assistant for question-answe...,The Heisenberg uncertainty principle states th...,The Heisenberg uncertainty principle is a fund...,2.242661,170,77,NaN,3.517278,170,77,NaN,The Heisenberg uncertainty principle states th...
1,Human: You are an assistant for question-answe...,The Schrödinger equation is a fundamental equa...,The Schrödinger equation is a fundamental equa...,1.525827,661,90,NaN,4.933964,661,117,NaN,The Schrödinger equation is a fundamental equa...
2,Human: You are an assistant for question-answe...,The greenhouse effect is a natural process tha...,The greenhouse effect is a natural process whe...,2.249457,604,138,NaN,2.814772,604,99,NaN,The greenhouse effect is a natural process tha...
3,Human: You are an assistant for question-answe...,The photoelectric effect is a phenomenon in wh...,The photoelectric effect is a phenomenon where...,1.584295,588,117,NaN,2.846362,588,120,NaN,"When light shines on a metal, electrons can be..."
4,Human: You are an assistant for question-answe...,The structure of the atom was determined throu...,The structure of the atom was determined throu...,2.418055,582,158,NaN,5.651367,582,129,NaN,"Modern atomic models, based on quantum mechani..."
5,Human: You are an assistant for question-answe...,Catalysts play a crucial role in chemical reac...,The role of catalysts in chemical reactions is...,1.296257,544,66,NaN,2.267321,544,85,NaN,A catalyst is a substance that can be added to...
6,Human: You are an assistant for question-answe...,The second law of thermodynamics states that i...,The second law of thermodynamics states that i...,1.807585,599,109,NaN,2.683165,599,107,NaN,The second law of thermodynamics states that t...
7,Human: You are an assistant for question-answe...,```\nThe main difference between nuclear fissi...,The main difference between nuclear fission an...,1.808980,472,97,NaN,2.327902,472,75,NaN,The phenomenon of nuclear fission. Fission occ...
8,Human: You are an assistant for question-answe...,The main differences between classical mechani...,The main differences between classical mechani...,1.655280,702,63,NaN,2.043752,702,94,NaN,Classical mechanics describes the physics of m...
9,Human: You are an assistant for question-answe...,The main difference between endothermic and ex...,The main difference between endothermic and ex...,1.802849,679,86,NaN,1.750192,679,75,NaN,If you touch a container that holds an endothe...


### Construct the ***LLM as a Judge Prompt Template***
---

In this portion of the notebook, the prompt template that is used by the LLM as a judge is prepared. This sample contains examples of evaluation prompt templates using a Llama3 evaluation prompt template [here](model-evals/llm_as_a_judge/data/prompt_template/llama3_eval_prompt.txt). There is another example of an Anthropic Claude Evaluation prompt template [here](model-evals/llm_as_a_judge/data/prompt_template/claude_eval_prompt.txt).

Information on which LLM as a judge to use can be configured in the `llm_as_a_judge_info` section of the config file.

In [8]:
if df_resp_all is not None:
    df_resp_all['eval_prompt'] = df_resp_all.apply(lambda r: prepare_eval_prompts(r), axis=1)
    logger.info("preparing the evaluation prompt templates for the LLM judge....")
else:
    logger.error(f"Model evaluation dataset is not available to process.")
eval_path_df: str = os.path.join(METRICS_DIR, config['dir_info']['processed_eval_prompts'])
df_resp_all.insert(0, 'prompt_id', df_resp_all.index)
df_resp_all.to_csv(eval_path_df, index=False)

[2024-06-08 11:06:04,953] p23556 {3998410777.py:13} INFO - evaluation prompt template recorded: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Use the following pieces of retrieved context in the section demarcated by "```" and the question related to that task below it. There are responses from different models completing that task by answering the question below. Your task is to select the answer that best answers the question based on the task provided in terms of relevancy and correctness.
Put the selected answer (without truncating it and give the complete answer within your response), model name and explanation for selecting the answer and not selecting other answer in a JSON as within 3 elements: "best_match_answer" (which contains the full answer you select), "selected_model" (which contains the model name), and "explanation". 
Your explanation should include both model name and answer description so that it is simple to understand which answer was generated by whic

model_responses: ['\n<anthropic.claude-3-haiku-20240307-v1:0>\nThe Heisenberg uncertainty principle states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously. This principle arises from the wave-particle duality of quantum particles and has profound implications for our understanding of the behavior of matter at the atomic and subatomic scales.\n</anthropic.claude-3-haiku-20240307-v1:0>\n', '\n<anthropic.claude-3-sonnet-20240229-v1:0>\nThe Heisenberg uncertainty principle is a fundamental principle in quantum mechanics that states there is a limit to how precisely certain pairs of physical properties of a particle, such as position and momentum, can be measured simultaneously. It arises from the wave-particle duality of quantum particles and has profound implications for understanding the behavior of matter at atomic and subatomic scales.\n</anthropic.claude-3-sonne

In [9]:
df_resp_all

,prompt_id,user_input,anthropic.claude-3-haiku-20240307-v1:0-response,anthropic.claude-3-sonnet-20240229-v1:0-response,anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds,anthropic.claude-3-haiku-20240307-v1:0-prompt_token_count,anthropic.claude-3-haiku-20240307-v1:0-completion_token_count,anthropic.claude-3-haiku-20240307-v1:0-exception,anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds,anthropic.claude-3-sonnet-20240229-v1:0-prompt_token_count,anthropic.claude-3-sonnet-20240229-v1:0-completion_token_count,anthropic.claude-3-sonnet-20240229-v1:0-exception,model_1,eval_prompt
0,0,Human: You are an assistant for question-answe...,The Heisenberg uncertainty principle states th...,The Heisenberg uncertainty principle is a fund...,2.242661,170,77,NaN,3.517278,170,77,NaN,The Heisenberg uncertainty principle states th...,<|begin_of_text|><|start_header_id|>user<|end_...
1,1,Human: You are an assistant for question-answe...,The Schrödinger equation is a fundamental equa...,The Schrödinger equation is a fundamental equa...,1.525827,661,90,NaN,4.933964,661,117,NaN,The Schrödinger equation is a fundamental equa...,<|begin_of_text|><|start_header_id|>user<|end_...
2,2,Human: You are an assistant for question-answe...,The greenhouse effect is a natural process tha...,The greenhouse effect is a natural process whe...,2.249457,604,138,NaN,2.814772,604,99,NaN,The greenhouse effect is a natural process tha...,<|begin_of_text|><|start_header_id|>user<|end_...
3,3,Human: You are an assistant for question-answe...,The photoelectric effect is a phenomenon in wh...,The photoelectric effect is a phenomenon where...,1.584295,588,117,NaN,2.846362,588,120,NaN,"When light shines on a metal, electrons can be...",<|begin_of_text|><|start_header_id|>user<|end_...
4,4,Human: You are an assistant for question-answe...,The structure of the atom was determined throu...,The structure of the atom was determined throu...,2.418055,582,158,NaN,5.651367,582,129,NaN,"Modern atomic models, based on quantum mechani...",<|begin_of_text|><|start_header_id|>user<|end_...
5,5,Human: You are an assistant for question-answe...,Catalysts play a crucial role in chemical reac...,The role of catalysts in chemical reactions is...,1.296257,544,66,NaN,2.267321,544,85,NaN,A catalyst is a substance that can be added to...,<|begin_of_text|><|start_header_id|>user<|end_...
6,6,Human: You are an assistant for question-answe...,The second law of thermodynamics states that i...,The second law of thermodynamics states that i...,1.807585,599,109,NaN,2.683165,599,107,NaN,The second law of thermodynamics states that t...,<|begin_of_text|><|start_header_id|>user<|end_...
7,7,Human: You are an assistant for question-answe...,```\nThe main difference between nuclear fissi...,The main difference between nuclear fission an...,1.808980,472,97,NaN,2.327902,472,75,NaN,The phenomenon of nuclear fission. Fission occ...,<|begin_of_text|><|start_header_id|>user<|end_...
8,8,Human: You are an assistant for question-answe...,The main differences between classical mechani...,The main differences between classical mechani...,1.655280,702,63,NaN,2.043752,702,94,NaN,Classical mechanics describes the physics of m...,<|begin_of_text|><|start_header_id|>user<|end_...
9,9,Human: You are an assistant for question-answe...,The main difference between endothermic and ex...,The main difference between endothermic and ex...,1.802849,679,86,NaN,1.750192,679,75,NaN,If you touch a container that holds an endothe...,<|begin_of_text|><|start_header_id|>user<|end_...


### Using LLM as a judge in the loop to evaluate and narrow down the responses generated by different models of choice

In [10]:
def llm_judge_json_evaluations(model_id: str, prompt: str):
    # represents the service name
    service_name: str = "bedrock"
    # represents creating the bedrock model to invoke the litellm api for response for titan, llama and claude
    bedrock_model: str = f"{service_name}/{model_id}"
    # represents the current aws region
    aws_region = boto3.Session().region_name 
    # initialize the response dict
    ret = dict(exception = None,
               user_prompt=None,
               prompt = prompt,
               completion = None,
               # initializing to 0 since none type throws an error later, this is used to calculate price per token input/output on ODT pricing
               completion_token_count = 0,
               # initializing to 0 since none type throws an error later
               prompt_token_count=0,
               input_token_cost = None, 
               output_token_cost = None,
               model_id = model_id)
    
    body = ret['prompt']
    os.environ["AWS_REGION_NAME"] = aws_region
    parameters = config['inference_parameters']
    temperature = parameters.get('temperature', 0.1)
    caching = parameters.get('caching', False)
    max_tokens = parameters.get("max_tokens", 500)

    try:
        # Represents calling the litellm completion/messaging api utilizing the completion/embeddings API
        logger.info(f"Invoking {bedrock_model}......")
        response = completion(model=bedrock_model,
                              messages=[{ "content": body,"role": "user"}],
                              temperature=temperature,
                              max_tokens=max_tokens,
                              caching=caching)
        # iterate through the entire model response
        for idx, choice in enumerate(response.choices):
            # extract the message and the message's content from litellm
            if choice.message and choice.message.content:
                # extract the response from the dict
                ret["completion"] = choice.message.content.strip()
        # Extract number of input and completion prompt tokens (this is the same structure for embeddings and text generation models on Amazon Bedrock)
        ret['prompt_token_count'] = response.usage.prompt_tokens
        ret['completion_token_count'] = response.usage.completion_tokens
        
    except Exception as e:
        logger.error(f"Exception occurred during invoking {model_id}, exception={e}")
        ret['exception'] = e
    logger.info(f"completion: {ret['completion']}")
    return ret

In [11]:
def get_inference(i: int, row: Dict, total: int, model_info: Dict) -> Dict:
    # save all the responses from the model in a dictionary
    resp: Dict = {}
    print(f"row={row}")
    model_id = model_info['model']
    # create the payload for model inference
    prompt = row['eval_prompt']
    # generate the chapter title based on the given chapter in the prompt 
    resp = llm_judge_json_evaluations(model_id, prompt)
    resp[config['dataset_info']['pre_existing_response_col']] = row[config['dataset_info']['pre_existing_response_col']]
    # calculate the input and output token price for all of the calls
    resp['input_token_cost'] = (resp['prompt_token_count']/1000) * model_info['input_tokens_pricing']
    resp['output_token_cost'] = (resp['completion_token_count']/1000) * model_info['output_tokens_pricing']
    dir_path = os.path.join(config['dir_info']['llm_as_a_judge_dir'], str(row['prompt_id']), model_id.replace(":", "-"))
    os.makedirs(dir_path, exist_ok=True)
    fpath = os.path.join(dir_path, f"model_evaluation_{row['prompt_id']}.json")
    logger.info(f"writing response={resp} to {fpath}")
    Path(fpath).write_text(json.dumps(resp, default=str, indent=2))
    logger.info(f"response {i}: {resp}")
    return resp

@ray.remote
def async_get_inference(i: int, row: Dict, total: int, model_info: Dict) -> Dict:
    logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
    return get_inference(i, row, total, model_info)

In [12]:
df_resp_all = json.loads(df_resp_all.to_json(orient='records'))
n: int = config.get('parallel_inference_count')
resp_list: List = []
erroneous_count = 0  # To keep track of errors
st = time.perf_counter()
EVAL_MODEL_INFO: Dict = config['llm_as_a_judge_info']
logger.info(f"------ running inference for {EVAL_MODEL_INFO.get('model')} -----")

# Split the input list
list_of_lists = [df_resp_all[i * n:(i + 1) * n] for i in range((len(df_resp_all) + n - 1) // n)]
logger.info(f"split input list of size {len(df_resp_all)} into {len(list_of_lists)} lists")

# Process each list
for idx, l in enumerate(list_of_lists):
    try:
        logger.info(f"getting inference for list {idx+1}/{len(list_of_lists)}, size of list={len(l)}")
        resp_list.extend(ray.get([async_get_inference.remote(i + 1, e, len(l), EVAL_MODEL_INFO) for i, e in enumerate(l)]))
    except Exception as e:
        logger.error(f"Error processing list {idx+1}/{len(list_of_lists)}: {e}")
        erroneous_count += 1

elapsed_time = time.perf_counter() - st
logger.info(f"------ model={EVAL_MODEL_INFO.get('model')} completed in {elapsed_time} ------")
logger.info(f"Total erroneous lists: {erroneous_count}")

[2024-06-08 11:06:05,328] p23556 {832808186.py:7} INFO - ------ running inference for meta.llama3-70b-instruct-v1:0 -----
[2024-06-08 11:06:05,330] p23556 {832808186.py:11} INFO - split input list of size 10 into 2 lists
[2024-06-08 11:06:05,332] p23556 {832808186.py:16} INFO - getting inference for list 1/2, size of list=5
(async_get_inference pid=14012) [2024-06-08 11:06:11,497] p14012 {2911495111.py:30} INFO - Invoking bedrock/meta.llama3-70b-instruct-v1:0......
(async_get_inference pid=9688) [2024-06-08 11:06:11,532] p9688 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


(async_get_inference pid=14012) row={'prompt_id': 3, 'user_input': 'Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context in the section demarcated by "```" to answer the question. If you don\'t know the answer just say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\n```\nThe photoelectric effect is a phenomenon in which electrons are emitted from a metal surface when light of sufficient energy (above a certain threshold frequency) shines on it. This effect was first observed by Heinrich Hertz in 1887 and later explained by Albert Einstein in 1905, for which he received the Nobel Prize in Physics in 1921.\n\nThe photoelectric effect contradicted classical physics, which predicted that the energy of the emitted electrons should increase with the intensity of the incident light. However, experiments showed that the energy of the emitted electrons depended only on the frequency of the light and not on its 

(async_get_inference pid=14012) 11:06:12 - LiteLLM:INFO: utils.py:992 - 
(async_get_inference pid=14012) Request Sent from LiteLLM:
(async_get_inference pid=14012) 
(async_get_inference pid=14012)             response = client.invoke_model(
(async_get_inference pid=14012)                 body={"prompt": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nUse the following pieces of retrieved context in the section demarcated by \"```\" and the question related to that task below it. There are responses from different models completing that task by answering the question below. Your task is to select the answer that best answers the question based on the task provided in terms of relevancy and correctness.\nPut the selected answer (without truncating it and give the complete answer within your response), model name and explanation for selecting the answer and not selecting other answer in a JSON as within 3 elements: \"best_match_answer\" (which contains the full answer you se

(async_get_inference pid=14012) row={'prompt_id': 6, 'user_input': 'Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context in the section demarcated by "```" to answer the question. If you don\'t know the answer just say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\n```\nThe second law of thermodynamics is one of the fundamental laws of physics that governs the behavior of energy and its transformation in thermodynamic systems. It is closely related to the concept of entropy, which is a measure of disorder or randomness in a system.\n\nThe second law states that in an isolated system, the total entropy always increases over time, and the process is irreversible. This means that in a closed system, energy tends to disperse and become more disordered, and it is impossible to transfer heat from a cooler object to a hotter object without doing work.\n\nThe mathematical expression of the second law is:\n\nΔ

(async_get_inference pid=19740) [2024-06-08 11:06:25,513] p19740 {2911495111.py:49} INFO - completion: {"best_match_answer": "Catalysts play a crucial role in chemical reactions by providing an alternative pathway with a lower activation energy, which allows the reaction to proceed more easily and at a faster rate. They do this by interacting with the reactants and forming intermediate species that require less energy to undergo the necessary bond rearrangements.", "selected_model": "anthropic.claude-3-haiku-20240307-v1:0", "explanation": "This answer is selected because it directly answers the question and provides a clear explanation of the role of catalysts in chemical reactions. The other models' answers are either too brief or rephrase the question, whereas this model provides a concise and accurate description of the catalyst's role."}
(async_get_inference pid=19740) [2024-06-08 11:06:25,514] p19740 {644234907.py:17} INFO - writing response={'exception': None, 'user_prompt': None

In [13]:
# view the raw responses from the LLM as a judge evaluation
df_resp_all

(async_get_inference pid=9688) 11:06:27 - LiteLLM:INFO: utils.py:2708 - Wrapper: Completed Call, calling success_handler
(async_get_inference pid=9688) [2024-06-08 11:06:27,015] p9688 {utils.py:2708} INFO - Wrapper: Completed Call, calling success_handler
(async_get_inference pid=9688) [2024-06-08 11:06:27,017] p9688 {2911495111.py:49} INFO - completion: {"best_match_answer": "The main differences between classical mechanics and quantum mechanics are: 1. Scale: Classical mechanics applies to macroscopic objects, while quantum mechanics deals with atomic and subatomic scales. 2. Determinism: Classical mechanics is deterministic, while quantum mechanics introduces probabilistic behavior and uncertainty. 3. Particle behavior: Classical mechanics treats particles as localized objects, while quantum mechanics describes particles as wave-like entities with probabilistic behavior.", "selected_model": "anthropic.claude-3-sonnet-20240229-v1:0", "explanation": "This model provides a clear and co

[{'prompt_id': 0,
  'user_input': 'Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context in the section demarcated by "```" to answer the question. If you don\'t know the answer just say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\n```\nThe Heisenberg uncertainty principle is a fundamental principle in quantum mechanics that states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously. This principle arises from the wave-particle duality of quantum particles and has profound implications for our understanding of the behavior of matter at the atomic and subatomic scales\n```\n\nQuestion: What is the Heisenberg uncertainty principle?\n\nAssistant:',
  'anthropic.claude-3-haiku-20240307-v1:0-response': 'The Heisenberg uncertainty principle states that there is a fundamental limit to the

### Visualize `LLM as a judge` completions and get more evaluation metrics

In [14]:
## Represents extracted all metric files
fpath_evaluated_files = os.path.join(config['dir_info']['llm_as_a_judge_dir'], "**", "*", "*.json")
eval_metric_files = glob.glob(fpath_evaluated_files, recursive=True)
logger.info(f"there are {len(eval_metric_files)} evaluated files by {config['llm_as_a_judge_info']['model']} LLM judge in {fpath_evaluated_files}")

[2024-06-08 11:06:27,161] p23556 {3305916361.py:4} INFO - there are 10 evaluated files by meta.llama3-70b-instruct-v1:0 LLM judge in eval_completions\**\*\*.json


In [15]:
def extract_sections(text: str) -> Optional[str]:
    """
    This function is used to clean up the data generated by the LLM as a judge to get
    responses split out a JSON format
    """
    try:
        question_match = re.search(r'Question:(.*?)```', text, re.DOTALL)
        question = question_match.group(1).strip() if question_match else None
    except Exception as e:
        print(f"The question was not extracted: {e}")
        question = None
    return question

In [16]:
os.makedirs(config['dir_info']['metrics'], exist_ok=True)
model_evaluation_responses = []

for f in eval_metric_files:
    with open(f, 'r') as file:
        model_evaluation_responses.append(json.loads(file.read()))
# results_df will contain the evaluation responses, including the completion and the model id
results_df = pd.DataFrame(model_evaluation_responses)
raw_llm_as_a_judge_responses: str = config['dir_info']['raw_llm_as_a_judge_completions']
raw_llm_fpath: str = os.path.join(METRICS_DIR, raw_llm_as_a_judge_responses)
results_df = results_df.dropna(axis=1, how='all')
results_df.head(10)

,prompt,completion,completion_token_count,prompt_token_count,input_token_cost,output_token_cost,model_id,model_1
0,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The Heisenberg uncertai...",161,894,0.002369,0.000563,meta.llama3-70b-instruct-v1:0,The Heisenberg uncertainty principle states th...
1,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The Schrödinger equatio...",161,1452,0.003848,0.000563,meta.llama3-70b-instruct-v1:0,The Schrödinger equation is a fundamental equa...
2,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The greenhouse effect i...",219,1375,0.003644,0.000767,meta.llama3-70b-instruct-v1:0,The greenhouse effect is a natural process tha...
3,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The photoelectric effec...",207,1366,0.003620,0.000724,meta.llama3-70b-instruct-v1:0,"When light shines on a metal, electrons can be..."
4,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The structure of the at...",240,1497,0.003967,0.000840,meta.llama3-70b-instruct-v1:0,"Modern atomic models, based on quantum mechani..."
5,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""Catalysts play a crucia...",152,1253,0.003320,0.000532,meta.llama3-70b-instruct-v1:0,A catalyst is a substance that can be added to...
6,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The second law of therm...",195,1336,0.003540,0.000683,meta.llama3-70b-instruct-v1:0,The second law of thermodynamics states that t...
7,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The main difference bet...",178,1172,0.003106,0.000623,meta.llama3-70b-instruct-v1:0,The phenomenon of nuclear fission. Fission occ...
8,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The main differences be...",163,1379,0.003654,0.000571,meta.llama3-70b-instruct-v1:0,Classical mechanics describes the physics of m...
9,<|begin_of_text|><|start_header_id|>user<|end_...,"{""best_match_answer"": ""The main difference bet...",192,1338,0.003546,0.000672,meta.llama3-70b-instruct-v1:0,If you touch a container that holds an endothe...


In [17]:
def replace_unescaped_quotes(pairs):
    new_pairs = []
    for key, value in pairs:
        if isinstance(value, str):
            value = value.replace("'", r"\'").replace('"', r'\"')
        new_pairs.append((key, value))
    return dict(new_pairs)

def clean_model_eval_json(data):
    """
    This function takes in JSON data, cleans it, and assigns the selected title as outputted by the model evaluator.
    """
    try:
        # Preprocess the input string to handle unescaped double quotes at the start
        if data.startswith('"'):
            data = "'" + data[1:-1].replace('"', '\\"') + "'"
        data = data.replace('\n', ' ')

        json_data = json.loads(data, object_pairs_hook=replace_unescaped_quotes)
        
        # Remove angle brackets from the selected_model value
        selected_model = json_data.get('selected_model', '')
        json_data['selected_model'] = re.sub(r'[<>]', '', selected_model)

        return pd.Series({
            'best_match_answer': json_data.get('best_match_answer'),
            'selected_model': json_data.get('selected_model'),
            'explanation': json_data.get('explanation'),
        })
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Invalid JSON data: {data} - {e}")
        return pd.Series({
            'best_match_answer': None,
            'selected_model': None,
            'explanation': None,
        })

In [18]:
def tidy_split(df, column, sep=',', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [19]:
new_results_df = results_df['completion'].apply(clean_model_eval_json)
# removing any unnecessary characters from the selected_model if any
new_results_df['selected_model'] = new_results_df['selected_model'].str.replace(r'<[^>]+>', '', regex=True)
# here we split the elements of the selected_model column using the tidy split function
new_exploded_df = tidy_split(new_results_df, 'selected_model', sep=',')
new_results_df[config['dataset_info']['pre_existing_response_col']] = results_df[config['dataset_info']['pre_existing_response_col']]
new_results_df['input_token_cost'] = results_df['input_token_cost']
new_results_df['output_token_cost'] = results_df['output_token_cost']
logger.info(f"All evaluation data is read into a dataframe of shape {results_df.shape}")
cols = new_results_df.columns.tolist()
idx = cols.index('selected_model')
cols.insert(idx + 1, cols.pop(cols.index(config['dataset_info']['pre_existing_response_col'])))
new_results_df.drop(columns=['input_token_cost', 'output_token_cost'], inplace=True)
# display the selected title, model explanation and the respective golden title in a side by side view
new_results_df.head(20)

[2024-06-08 11:06:27,458] p23556 {471329066.py:9} INFO - All evaluation data is read into a dataframe of shape (10, 8)


,best_match_answer,selected_model,explanation,model_1
0,The Heisenberg uncertainty principle states th...,anthropic.claude-3-haiku-20240307-v1:0,I selected this answer because it provides a c...,The Heisenberg uncertainty principle states th...
1,The Schrödinger equation is a fundamental equa...,model_1,I selected model_1 because it provides a conci...,The Schrödinger equation is a fundamental equa...
2,The greenhouse effect is a natural process tha...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The greenhouse effect is a natural process tha...
3,The photoelectric effect is a phenomenon in wh...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,"When light shines on a metal, electrons can be..."
4,The structure of the atom was determined throu...,anthropic.claude-3-haiku-20240307-v1:0,This answer was selected because it provides a...,"Modern atomic models, based on quantum mechani..."
5,Catalysts play a crucial role in chemical reac...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,A catalyst is a substance that can be added to...
6,The second law of thermodynamics states that i...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The second law of thermodynamics states that t...
7,The main difference between nuclear fission an...,anthropic.claude-3-haiku-20240307-v1:0,This model was selected because it provides a ...,The phenomenon of nuclear fission. Fission occ...
8,The main differences between classical mechani...,anthropic.claude-3-sonnet-20240229-v1:0,This model provides a clear and concise answer...,Classical mechanics describes the physics of m...
9,The main difference between endothermic and ex...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,If you touch a container that holds an endothe...


In [20]:
initial_df = pd.read_csv(eval_path_df)
# Merge the two DataFrames on 'gpt_response'
merged_df = pd.merge(new_results_df, initial_df[[config['dataset_info']['pre_existing_response_col'], 
                                                config['dataset_info']['user_question_col']]], on=config['dataset_info']['pre_existing_response_col'], how='left')

cols = [col for col in merged_df.columns if col != 'user prompt']
processed_prompts_for_eval_path = os.path.join(METRICS_DIR, config['dir_info']['llm_as_a_judge_comparisons'])
merged_df.to_csv(processed_prompts_for_eval_path, index=False)
merged_df

,best_match_answer,selected_model,explanation,model_1,user_input
0,The Heisenberg uncertainty principle states th...,anthropic.claude-3-haiku-20240307-v1:0,I selected this answer because it provides a c...,The Heisenberg uncertainty principle states th...,Human: You are an assistant for question-answe...
1,The Schrödinger equation is a fundamental equa...,model_1,I selected model_1 because it provides a conci...,The Schrödinger equation is a fundamental equa...,Human: You are an assistant for question-answe...
2,The greenhouse effect is a natural process tha...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The greenhouse effect is a natural process tha...,Human: You are an assistant for question-answe...
3,The photoelectric effect is a phenomenon in wh...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,"When light shines on a metal, electrons can be...",Human: You are an assistant for question-answe...
4,The structure of the atom was determined throu...,anthropic.claude-3-haiku-20240307-v1:0,This answer was selected because it provides a...,"Modern atomic models, based on quantum mechani...",Human: You are an assistant for question-answe...
5,Catalysts play a crucial role in chemical reac...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,A catalyst is a substance that can be added to...,Human: You are an assistant for question-answe...
6,The second law of thermodynamics states that i...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The second law of thermodynamics states that t...,Human: You are an assistant for question-answe...
7,The main difference between nuclear fission an...,anthropic.claude-3-haiku-20240307-v1:0,This model was selected because it provides a ...,The phenomenon of nuclear fission. Fission occ...,Human: You are an assistant for question-answe...
8,The main differences between classical mechani...,anthropic.claude-3-sonnet-20240229-v1:0,This model provides a clear and concise answer...,Classical mechanics describes the physics of m...,Human: You are an assistant for question-answe...
9,The main difference between endothermic and ex...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,If you touch a container that holds an endothe...,Human: You are an assistant for question-answe...


### View the LLM as a judge comparison and evaluation

In [21]:
processed_prompts_for_eval_path = os.path.join(METRICS_DIR, config['dir_info']['llm_as_a_judge_comparisons'])
merged_df = pd.read_csv(processed_prompts_for_eval_path)
merged_df

,best_match_answer,selected_model,explanation,model_1,user_input
0,The Heisenberg uncertainty principle states th...,anthropic.claude-3-haiku-20240307-v1:0,I selected this answer because it provides a c...,The Heisenberg uncertainty principle states th...,Human: You are an assistant for question-answe...
1,The Schrödinger equation is a fundamental equa...,model_1,I selected model_1 because it provides a conci...,The Schrödinger equation is a fundamental equa...,Human: You are an assistant for question-answe...
2,The greenhouse effect is a natural process tha...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The greenhouse effect is a natural process tha...,Human: You are an assistant for question-answe...
3,The photoelectric effect is a phenomenon in wh...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,"When light shines on a metal, electrons can be...",Human: You are an assistant for question-answe...
4,The structure of the atom was determined throu...,anthropic.claude-3-haiku-20240307-v1:0,This answer was selected because it provides a...,"Modern atomic models, based on quantum mechani...",Human: You are an assistant for question-answe...
5,Catalysts play a crucial role in chemical reac...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,A catalyst is a substance that can be added to...,Human: You are an assistant for question-answe...
6,The second law of thermodynamics states that i...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The second law of thermodynamics states that t...,Human: You are an assistant for question-answe...
7,The main difference between nuclear fission an...,anthropic.claude-3-haiku-20240307-v1:0,This model was selected because it provides a ...,The phenomenon of nuclear fission. Fission occ...,Human: You are an assistant for question-answe...
8,The main differences between classical mechani...,anthropic.claude-3-sonnet-20240229-v1:0,This model provides a clear and concise answer...,Classical mechanics describes the physics of m...,Human: You are an assistant for question-answe...
9,The main difference between endothermic and ex...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,If you touch a container that holds an endothe...,Human: You are an assistant for question-answe...


In [22]:
# Convert the DataFrame to JSON
merged_df_json = merged_df.to_json(orient='records')

# Save the JSON to a text file
with open(JSON_TXT_FILE_PATH, 'w') as json_text_file:
    json_text_file.write(merged_df_json)
logger.info(f"CSV saved to: {processed_prompts_for_eval_path}")

[2024-06-08 11:06:27,643] p23556 {1042489399.py:7} INFO - CSV saved to: data\results\llm_as_a_judge_comparisons.csv


### Generate the LLM as a judge `pick rate` to show how many times a model was picked having the best response over the other models

In [23]:
# Compute the percentage of each model selection and reset the index
new_exploded_df['selected_model'] = new_exploded_df['selected_model'].map(lambda x: x.strip())
response_index_percentage_df = new_exploded_df['selected_model'].value_counts(normalize=True).reset_index()
response_distribution_fpath = os.path.join(METRICS_DIR, config['dir_info']['llm_as_a_judge_pick_rate'])
response_index_percentage_df['proportion'] *= 100
response_index_percentage_df.to_csv(response_distribution_fpath, index=False)
response_index_percentage_df.head(10)

,selected_model,proportion
0,anthropic.claude-3-haiku-20240307-v1:0,80.0
1,model_1,10.0
2,anthropic.claude-3-sonnet-20240229-v1:0,10.0


### Final Summary: `LLM evaluation`

In [24]:
# simple function to get a final summary on all of the data provided from LLM as a judge
def final_analysis_summary(bedrock: botocore.client, 
                           prompt: str) -> str:
    """
    This function takes in the prompt that checks whether the text file has a response to the question and if not, 
    returns "not found" to move to the next hit
    """
    modelId=FINAL_ANALYSIS_MODEL_ID
    body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2000,
        "temperature": 0.1,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
    })

    try:
        response = bedrock.invoke_model(
        modelId=modelId,
        body=body)

        response_body = json.loads(response['body'].read().decode("utf-8"))
        llm_response = response_body['content'][0]['text'].replace('"', "'")

    except Exception as e:
        logger.error(f"exception={e}")
        llm_response = None
    return llm_response

In [25]:
new_results_df

,best_match_answer,selected_model,explanation,model_1
0,The Heisenberg uncertainty principle states th...,anthropic.claude-3-haiku-20240307-v1:0,I selected this answer because it provides a c...,The Heisenberg uncertainty principle states th...
1,The Schrödinger equation is a fundamental equa...,model_1,I selected model_1 because it provides a conci...,The Schrödinger equation is a fundamental equa...
2,The greenhouse effect is a natural process tha...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The greenhouse effect is a natural process tha...
3,The photoelectric effect is a phenomenon in wh...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,"When light shines on a metal, electrons can be..."
4,The structure of the atom was determined throu...,anthropic.claude-3-haiku-20240307-v1:0,This answer was selected because it provides a...,"Modern atomic models, based on quantum mechani..."
5,Catalysts play a crucial role in chemical reac...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,A catalyst is a substance that can be added to...
6,The second law of thermodynamics states that i...,anthropic.claude-3-haiku-20240307-v1:0,This model provides a clear and concise explan...,The second law of thermodynamics states that t...
7,The main difference between nuclear fission an...,anthropic.claude-3-haiku-20240307-v1:0,This model was selected because it provides a ...,The phenomenon of nuclear fission. Fission occ...
8,The main differences between classical mechani...,anthropic.claude-3-sonnet-20240229-v1:0,This model provides a clear and concise answer...,Classical mechanics describes the physics of m...
9,The main difference between endothermic and ex...,anthropic.claude-3-haiku-20240307-v1:0,This answer is selected because it directly an...,If you touch a container that holds an endothe...


In [26]:
with open(ALL_EXPLANATIONS_FPATH, 'w') as file:
    for index, row in merged_df.iterrows():
        file.write(f"Selected Model: {row['selected_model']}\nExplanation: {row['explanation']}\n\n")

# Read the content back to use as analysis context
with open(ALL_EXPLANATIONS_FPATH, 'r') as file:
    analysis_context = file.read()
print(analysis_context)

Selected Model: anthropic.claude-3-haiku-20240307-v1:0
Explanation: I selected this answer because it provides a complete and accurate description of the Heisenberg uncertainty principle, including its implications. The other models provided shorter or slightly rephrased versions of the principle, but this one is the most comprehensive and faithful to the original context.

Selected Model: model_1
Explanation: I selected model_1 because it provides a concise and clear answer that covers the definition and application of the Schrödinger equation. The other models provide similar information, but model_1\'s answer is more comprehensive and directly addresses the question.

Selected Model: anthropic.claude-3-haiku-20240307-v1:0
Explanation: This model provides a clear and concise explanation of the greenhouse effect, its importance, and how human activities have led to an enhanced greenhouse effect and global warming. The other models either provide a partial explanation or lack detail, m

In [27]:
# open the prompt template and prepare it for inference
with open(config['dir_info']['claude_final_summary_eval_prompt'], 'r') as file:
    final_summary_prompt = file.read()
    processed_summary_eval_prompt: str = final_summary_prompt.format(context=analysis_context)

endpoint_url: str = config['bedrock_ep_url'].format(region=config['aws']['region'])
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=endpoint_url)
final_analysis: str = final_analysis_summary(bedrock, prompt=processed_summary_eval_prompt)

[2024-06-08 11:06:27,845] p23556 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [30]:
logger.info(final_analysis)

[2024-06-08 11:06:55,578] p23556 {4102048802.py:1} INFO - Based on the context provided, the model anthropic.claude-3-haiku-20240307-v1:0 was selected more frequently than the other models. The key reasons for its selection were:

1. Comprehensive and accurate explanations: This model consistently provided clear, concise, and complete explanations of various scientific concepts and principles. For example, in the case of the Heisenberg uncertainty principle, it was chosen for providing a 'complete and accurate description' that was faithful to the original context.

2. Clarity and conciseness: The explanations given by this model were praised for their clarity and conciseness. It effectively summarized the main points without including extraneous information. For instance, in explaining the photoelectric effect, it was selected for providing a 'clear and concise explanation' of the concept and its significance.

3. Addressing the question directly: In several cases, this model was chos

In [29]:
Path(FINAL_SUMMARY_ANALYSIS).write_text(final_analysis + "\n")

1798